In [23]:
from biopandas.pdb import PandasPdb
from itertools import chain
from Bio import PDB
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

from sklearn.cluster import KMeans

import warnings
warnings.simplefilter(action="ignore")

# 0. TOOLS

In [14]:
def residue_remove(pdbfile, remove_ids, outputName):
    """
    Assumption: pdbfile only has A chain
    """
    residue_to_remove = []

    pdb_io = PDB.PDBIO()
    pdb_parser = PDB.PDBParser()
    structure = pdb_parser.get_structure(" ", pdbfile)

    model = structure[0]
    chain = model["A"]

    for residue in chain:
        id = residue.id
        if id[1] in remove_ids: 
            residue_to_remove.append(residue.id)

    for residue in residue_to_remove:
        chain.detach_child(residue)
    pdb_io.set_structure(structure)
    pdb_io.save(outputName)
    return 

In [84]:
def update_resnum(pdbfile):
    """
    🌟 CHANGE RESIDUE NUMBER
    Assumption: pdbfile only has A chain
    """
    pdb_io = PDB.PDBIO()
    pdb_parser = PDB.PDBParser()
    structure = pdb_parser.get_structure(" ", pdbfile)

    model = structure[0]
    chain = model["A"]
    print(len([i for i in chain.get_residues()]))
    new_resnums = list(range(1, 1+len([i for i in chain.get_residues()])))

    for i, residue in enumerate(chain.get_residues()):
        res_id = list(residue.id)
        res_id[1] = new_resnums[i]
        residue.id = tuple(res_id)

    pdb_io.set_structure(structure)
    pdb_io.save(pdbfile.split('pdb')[0] + '_update_resnum.pdb')
    return

#  1. Load target protein

In [2]:
target_pdb = PandasPdb().read_pdb('./target/AF-Q9H1U9-F1-model_v4.pdb')

In [4]:
target_df = target_pdb.df['ATOM']
target_df

,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,...,x_coord,y_coord,z_coord,occupancy,b_factor,blank_4,segment_id,element_symbol,charge,line_idx
0,ATOM,1,,N,,MET,,A,1,,...,50.382,-9.024,-50.566,1.0,35.97,,,N,NaN,70
1,ATOM,2,,CA,,MET,,A,1,,...,50.276,-7.868,-51.476,1.0,35.97,,,C,NaN,71
2,ATOM,3,,C,,MET,,A,1,,...,49.309,-6.908,-50.793,1.0,35.97,,,C,NaN,72
3,ATOM,4,,CB,,MET,,A,1,,...,49.838,-8.362,-52.871,1.0,35.97,,,C,NaN,73
4,ATOM,5,,O,,MET,,A,1,,...,48.126,-7.209,-50.783,1.0,35.97,,,O,NaN,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2372,ATOM,2373,,O,,ILE,,A,297,,...,-0.792,25.811,-25.379,1.0,78.01,,,O,NaN,2442
2373,ATOM,2374,,CG1,,ILE,,A,297,,...,-3.634,24.414,-22.032,1.0,78.01,,,C,NaN,2443
2374,ATOM,2375,,CG2,,ILE,,A,297,,...,-1.337,25.379,-22.401,1.0,78.01,,,C,NaN,2444
2375,ATOM,2376,,CD1,,ILE,,A,297,,...,-3.337,23.661,-20.729,1.0,78.01,,,C,NaN,2445


In [55]:
target_df.nunique()

record_name          1
atom_number       2377
blank_1              1
atom_name           37
alt_loc              1
residue_name        20
blank_2              1
chain_id             1
residue_number     297
insertion            1
blank_3              1
x_coord           2321
y_coord           2306
z_coord           2318
occupancy            1
b_factor           273
blank_4              1
segment_id           1
element_symbol       4
charge               0
line_idx          2377
dtype: int64

In [61]:
target_df.dtypes

record_name        object
atom_number         int32
blank_1            object
atom_name          object
alt_loc            object
residue_name       object
blank_2            object
chain_id           object
residue_number      int32
insertion          object
blank_3            object
x_coord           float64
y_coord           float64
z_coord           float64
occupancy         float64
b_factor          float64
blank_4            object
segment_id         object
element_symbol     object
charge            float64
line_idx            int64
dtype: object

# 2. Truncate target protein
## low pLDDT: 1-22
## transmembrane: 36-56, 85-105, 116-135, 179-199, 215-235, 268-289
## https://alphafold.ebi.ac.uk/entry/Q9H1U9
## https://www.uniprot.org/uniprotkb/Q9H1U9/entry#subcellular_location

In [32]:
truncating_ids = list(range(1,23))+list(range(36,57))+list(range(85,106))+list(range(116,136))+list(range(179,200))+list(range(215,236))+list(range(268,290))

In [56]:
297 - 148

149

In [54]:
len(truncating_ids)

148

In [33]:
residue_remove(pdbfile="./target/AF-Q9H1U9-F1-model_v4.pdb", 
               remove_ids=truncating_ids, 
               outputName="./target/truncation.pdb")

In [34]:
truncation = PandasPdb().read_pdb("./target/truncation.pdb")
cols = ["atom_name", "residue_name", "residue_number", "x_coord", "y_coord", "z_coord"]
df = truncation.df["ATOM"][cols]
df

,atom_name,residue_name,residue_number,x_coord,y_coord,z_coord
0,N,PRO,23,16.691,28.716,-21.862
1,CA,PRO,23,16.179,28.381,-20.521
2,C,PRO,23,14.687,28.705,-20.310
3,CB,PRO,23,17.085,29.147,-19.554
4,O,PRO,23,14.032,28.086,-19.476
...,...,...,...,...,...,...
1236,O,ILE,297,-0.792,25.811,-25.379
1237,CG1,ILE,297,-3.634,24.414,-22.032
1238,CG2,ILE,297,-1.337,25.379,-22.401
1239,CD1,ILE,297,-3.337,23.661,-20.729


In [57]:
len(set(df['residue_number'].tolist()))

149

In [68]:
# check: whether each resnum has one CB
resnum = list(set(df['residue_number'].tolist()))
CB_nan = []
i = 0
for res in resnum:
    sub_df = df[df["residue_number"]==res]
    if "CB" in sub_df["atom_name"].tolist():
        pass
    else:
        i += 1
        CB_nan.append(res)
        print(res)
print("i=", i)

29
62
76
83
168
170
175
244
245
i= 9


# 2. Visualize $fig$ truncation.pdb with only $C_{\beta}$ ATOM

In [35]:
df_CB = df[df["atom_name"]=="CB"]

In [50]:
len(set(df_CB['residue_number'].tolist()))

140

In [36]:
fig = go.Figure(data =[go.Scatter3d(x = df_CB['x_coord'],
                                    y = df_CB['y_coord'],
                                    z = df_CB['z_coord'],
                                    mode ='markers')])
fig.update_traces(marker_size=10)

fig.update_layout(
    width = 1100, 
    height=1100
)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

fig.update_layout(scene=dict(
    xaxis_title='x coord',
    yaxis_title='y coord',
    zaxis_title='z coord'
))
fig.show()

# 3. Grouping into 2 parts using clustering for separately desiging binders for each part 
## when observing 3D plot, we can see there are 2 clear distinguishing parts

In [37]:
x = df_CB[['x_coord', 'y_coord', 'z_coord']].values
model = KMeans(n_clusters = 2, init = "k-means++", max_iter = 300, n_init = 10, random_state = 0)
y_clusters = model.fit_predict(x)

df_CB['label'] = [str(i) for i in y_clusters]
df_CB

,atom_name,residue_name,residue_number,x_coord,y_coord,z_coord,label
3,CB,PRO,23,17.085,29.147,-19.554,1
10,CB,HIS,24,12.505,31.513,-21.284,1
20,CB,ILE,25,11.779,27.657,-25.057,1
28,CB,THR,26,13.504,24.063,-21.142,1
35,CB,ASN,27,11.041,27.148,-17.728,1
...,...,...,...,...,...,...,...
1203,CB,LEU,293,-5.104,19.547,-21.490,1
1211,CB,LEU,294,-1.227,18.292,-25.069,1
1219,CB,LYS,295,-4.821,19.749,-28.811,1
1228,CB,VAL,296,-7.003,23.205,-25.432,1


In [38]:
# cluster visualization
fig2 = px.scatter_3d(df_CB, x='x_coord', y='y_coord', z='z_coord', color='label')
fig2.update_traces(marker_size=10)

fig2.update_layout(
    width = 1100, 
    height= 1100
)
fig2.show()

# 4. Get transmembrane outside and inside protein pdb
### outside and inside protein residue number

In [64]:
out_resnum = df_CB[df_CB['label'] == '0']['residue_number'].tolist()
in_resnum = df_CB[df_CB['label'] == '1']['residue_number'].tolist()

In [66]:
len(out_resnum), len(in_resnum) # =140

(95, 45)

In [67]:
len(set(out_resnum)), len(set(in_resnum))

(95, 45)

### outside_pdb and inside_pdb

In [81]:
## OUTSIDE PROTEIN - label0
residue_remove(pdbfile="./target/truncation.pdb", 
               remove_ids=in_resnum+CB_nan, 
               outputName="./target/outside.pdb")

## INSIDE PROTEIN - label1
residue_remove(pdbfile="./target/truncation.pdb", 
               remove_ids=out_resnum+CB_nan, 
               outputName="./target/inside.pdb")

In [70]:
# for test
op = PandasPdb().read_pdb("./target/outside.pdb")
ip = PandasPdb().read_pdb("./target/inside.pdb")
len(set(op.df['ATOM']['residue_number'])), len(set(ip.df['ATOM']['residue_number']))

(95, 45)

# 5. Replace residue numbers by continuous numbers 🧬🌟 CHANGE RESIDUE NUMBER

In [86]:
## for whole 
update_resnum(pdbfile="./target/truncation.pdb")

## for outside part
update_resnum(pdbfile="./target/outside.pdb")

## for inside part
update_resnum(pdbfile="./target/inside.pdb")

149
95
45


# 6. Select specifi AA position 🤨 for only ouside_update_resnum.pdb
#### ref: https://www.sigmaaldrich.com/US/en/technical-documents/technical-article/protein-biology/protein-structural-analysis/amino-acid-reference-chart
#### ref: https://github.com/RosettaCommons/RFdiffusion
#### At pH $7^{B}$
### "In the paper we define a hotspot as a residue on the target protein which is within 10A Cbeta distance of the binder"

- Can I use AF2_multimer for validation? I found that use_multimer=True would help with the in
silico score.



In [88]:
very_hydrohobic = list(map(str.upper,['Leu', 'Ile', 'Phe', 'Trp', 'Val', 'Met']))
hydrohobic = list(map(str.upper,['Cys', 'Tyr', 'Ala']))
neural = list(map(str.upper,['Thr', 'His', 'Gly', 'Ser', 'Gln']))
hydrophilic = list(map(str.upper,['Arg', 'Lys', 'Asn', 'Glu', 'Pro', 'Asp']))

In [99]:
outside = PandasPdb().read_pdb('./target/outside_update_resnum.pdb')
outdf = outside.df["ATOM"][cols]
outdf_CB = outdf[outdf["atom_name"]=="CB"]
outdf_CB

,atom_name,residue_name,residue_number,x_coord,y_coord,z_coord
3,CB,ARG,1,-19.548,0.117,-0.729
14,CB,GLN,2,-17.363,-4.088,-3.220
23,CB,GLN,3,-12.863,-2.928,-0.537
32,CB,LEU,4,-15.576,-1.252,3.533
40,CB,TYR,5,-19.833,-4.008,3.599
...,...,...,...,...,...,...
800,CB,ARG,91,-14.250,-19.722,5.362
811,CB,LYS,92,-16.204,-18.073,0.034
820,CB,LEU,93,-10.963,-17.682,-2.362
828,CB,ILE,94,-15.660,-16.372,-4.713


In [98]:
# # highlight very_hydrohobic aas
# hyd = []
# for res in outdf_CB['residue_name']:
#     if res in very_hydrohobic:
#         hyd.append('yes')
#     else:
#         hyd.append('no')
        
# outdf_CB['hydrohobic'] = hyd
# outdf_CB

,atom_name,residue_name,residue_number,x_coord,y_coord,z_coord,hydrohobic
3,CB,ARG,1,-19.548,0.117,-0.729,no
14,CB,GLN,2,-17.363,-4.088,-3.220,no
23,CB,GLN,3,-12.863,-2.928,-0.537,no
32,CB,LEU,4,-15.576,-1.252,3.533,yes
40,CB,TYR,5,-19.833,-4.008,3.599,no
...,...,...,...,...,...,...,...
800,CB,ARG,91,-14.250,-19.722,5.362,no
811,CB,LYS,92,-16.204,-18.073,0.034,no
820,CB,LEU,93,-10.963,-17.682,-2.362,yes
828,CB,ILE,94,-15.660,-16.372,-4.713,yes


In [106]:
very_hydrohobic_df=outdf_CB[outdf_CB['residue_name'].isin(very_hydrohobic)]

fig2.add_trace(
    go.Scatter3d(x=very_hydrohobic_df['x_coord'],
                 y=very_hydrohobic_df['y_coord'],
                 z=very_hydrohobic_df['z_coord'],
                 marker_color='yellow',
                 mode='markers',
                 text=very_hydrohobic_df['residue_number'])
)


# Make a PLANE!!!

In [112]:
indf = df_CB[df_CB["label"]=="1"]
# group indf into 3
x_tmp = indf[['x_coord', 'y_coord', 'z_coord']].values
model_tmp = KMeans(n_clusters = 3, init = "k-means++", max_iter = 300, n_init = 10, random_state = 0)
y_clusters_tmp = model_tmp.fit_predict(x_tmp)

indf['new_label'] = [str(i) for i in y_clusters_tmp]
indf

,atom_name,residue_name,residue_number,x_coord,y_coord,z_coord,label,new_label
3,CB,PRO,23,17.085,29.147,-19.554,1,2
10,CB,HIS,24,12.505,31.513,-21.284,1,2
20,CB,ILE,25,11.779,27.657,-25.057,1,2
28,CB,THR,26,13.504,24.063,-21.142,1,2
35,CB,ASN,27,11.041,27.148,-17.728,1,2
43,CB,VAL,28,7.024,27.884,-21.446,1,2
54,CB,GLU,30,8.863,21.706,-17.761,1,2
63,CB,MET,31,5.113,25.341,-16.718,1,2
71,CB,LYS,32,2.711,23.394,-21.240,1,2
80,CB,HIS,33,4.251,18.522,-19.685,1,2


In [116]:
# cluster visualization
fig_tmp = px.scatter_3d(indf, x='x_coord', y='y_coord', z='z_coord', color='new_label')
fig_tmp.update_traces(marker_size=10)

fig_tmp.update_layout(
    width = 800, 
    height= 800
)
fig_tmp.show()

In [119]:
x0, y0, z0 = indf[indf["new_label"]=="0"][['x_coord', 'y_coord', 'z_coord']].mean().tolist()
x1, y1, z1 = indf[indf["new_label"]=="1"][['x_coord', 'y_coord', 'z_coord']].mean().tolist()
x2, y2, z2 = indf[indf["new_label"]=="2"][['x_coord', 'y_coord', 'z_coord']].mean().tolist()

In [120]:
fig2.add_trace(
    go.Scatter3d(x=[x0],
                 y=[y0],
                 z=[z0],
                 marker_color='green')
)
fig2.add_trace(
    go.Scatter3d(x=[x1],
                 y=[y1],
                 z=[z1],
                 marker_color='black')
)
fig2.add_trace(
    go.Scatter3d(x=[x2],
                 y=[y2],
                 z=[z2],
                 marker_color='orange')
)

In [123]:
pts =np.array([[x0, y0, z0],
              [x1, y1, z1],
              [x2, y2, z2]])
x, y, z = pts.T

fig2.add_trace(
    go.Mesh3d(x=x, y=y, z=z, color='lightpink', opacity=0.50)
)


In [125]:
outdf = df_CB[df_CB["label"]=="0"]
# group outdf into 3
x_tmp2 = outdf[['x_coord', 'y_coord', 'z_coord']].values
model_tmp2 = KMeans(n_clusters = 3, init = "k-means++", max_iter = 300, n_init = 10, random_state = 0)
y_clusters_tmp2 = model_tmp2.fit_predict(x_tmp2)

outdf['new_label2'] = [str(i) for i in y_clusters_tmp2]
outdf

,atom_name,residue_name,residue_number,x_coord,y_coord,z_coord,label,new_label2
110,CB,ARG,57,-19.548,0.117,-0.729,0,0
121,CB,GLN,58,-17.363,-4.088,-3.220,0,0
130,CB,GLN,59,-12.863,-2.928,-0.537,0,2
139,CB,LEU,60,-15.576,-1.252,3.533,0,0
147,CB,TYR,61,-19.833,-4.008,3.599,0,0
...,...,...,...,...,...,...,...,...
1127,CB,ARG,263,-14.250,-19.722,5.362,0,2
1138,CB,LYS,264,-16.204,-18.073,0.034,0,2
1147,CB,LEU,265,-10.963,-17.682,-2.362,0,2
1155,CB,ILE,266,-15.660,-16.372,-4.713,0,2


In [126]:
# cluster visualization
fig_tmp2 = px.scatter_3d(outdf, x='x_coord', y='y_coord', z='z_coord', color='new_label2')
fig_tmp2.update_traces(marker_size=10)

fig_tmp2.update_layout(
    width = 800, 
    height= 800
)
fig_tmp2.show()

In [128]:
x02, y02, z02 = outdf[outdf["new_label2"]=="0"][['x_coord', 'y_coord', 'z_coord']].mean().tolist()
x12, y12, z12 = outdf[outdf["new_label2"]=="1"][['x_coord', 'y_coord', 'z_coord']].mean().tolist()
x22, y22, z22 = outdf[outdf["new_label2"]=="2"][['x_coord', 'y_coord', 'z_coord']].mean().tolist()

In [129]:
fig2.add_trace(
    go.Scatter3d(x=[x02],
                 y=[y02],
                 z=[z02],
                 marker_color='green')
)
fig2.add_trace(
    go.Scatter3d(x=[x12],
                 y=[y12],
                 z=[z12],
                 marker_color='black')
)
fig2.add_trace(
    go.Scatter3d(x=[x22],
                 y=[y22],
                 z=[z22],
                 marker_color='orange')
)

In [132]:
pts2 =np.array([[x02, y02, z02],
              [x12, y12, z12],
              [x22, y22, z22]])
x2, y2, z2 = pts2.T

fig2.add_trace(
    go.Mesh3d(x=x2, y=y2, z=z2, color='lightgreen', opacity=0.50)
)


In [135]:
# distance calculation
def dist_to_plane(p, p0, p1, p2):
    u = p1 - p0
    v = p2 - p0
    # vector normal to plane
    n = np.cross(u, v)
    n /= np.linalg.norm(n)
    p_ = p - p0
    dist_to_plane = np.dot(p_, n)
    return dist_to_plane

In [155]:
outdf

,atom_name,residue_name,residue_number,x_coord,y_coord,z_coord,label,new_label2
110,CB,ARG,57,-19.548,0.117,-0.729,0,0
121,CB,GLN,58,-17.363,-4.088,-3.220,0,0
130,CB,GLN,59,-12.863,-2.928,-0.537,0,2
139,CB,LEU,60,-15.576,-1.252,3.533,0,0
147,CB,TYR,61,-19.833,-4.008,3.599,0,0
...,...,...,...,...,...,...,...,...
1127,CB,ARG,263,-14.250,-19.722,5.362,0,2
1138,CB,LYS,264,-16.204,-18.073,0.034,0,2
1147,CB,LEU,265,-10.963,-17.682,-2.362,0,2
1155,CB,ILE,266,-15.660,-16.372,-4.713,0,2


In [170]:
p0 = np.array([x02, y02, z02])
p1 = np.array([x12, y12, z12])
p2 = np.array([x22, y22, z22])
dis_dic = pd.DataFrame(columns=["x_coord","y_coord", "z_coord", "dis"])
dis_list, xl, yl, zl = [], [], [], []

In [173]:
p0 = np.array([x02, y02, z02])
p1 = np.array([x12, y12, z12])
p2 = np.array([x22, y22, z22])
dis_dic = pd.DataFrame(columns=["x_coord","y_coord", "z_coord", "dis"])
dis_list, xl, yl, zl = [], [], [], []

dis_list, xl, yl, zl = [], [], [], []
for i in range(0, len(outdf)):
    p = np.array(outdf.iloc[i][['x_coord', 'y_coord', 'z_coord']].tolist())
    dis = dist_to_plane(p, p0, p1, p2)
    dis_list.append(dis)
    xl.append(outdf.iloc[i]['x_coord'].tolist())
    yl.append(outdf.iloc[i]['y_coord'].tolist())
    zl.append(outdf.iloc[i]['z_coord'].tolist())
    

dis_dic["dis"] = dis_list
dis_dic["x_coord"] = xl
dis_dic["y_coord"] = yl
dis_dic["z_coord"] = zl
dis_dic["abs_dis"] = dis_dic['dis'].abs()

In [174]:
dis_dic

,x_coord,y_coord,z_coord,dis,abs_dis
0,-19.548,0.117,-0.729,1.589757,1.589757
1,-17.363,-4.088,-3.220,4.504881,4.504881
2,-12.863,-2.928,-0.537,5.043841,5.043841
3,-15.576,-1.252,3.533,0.321443,0.321443
4,-19.833,-4.008,3.599,-2.443185,2.443185
...,...,...,...,...,...
90,-14.250,-19.722,5.362,-2.013906,2.013906
91,-16.204,-18.073,0.034,1.312743,1.312743
92,-10.963,-17.682,-2.362,6.327248,6.327248
93,-15.660,-16.372,-4.713,5.622217,5.622217


In [156]:
dis_dic#.sort_values('abs_dis')

,resnum,dis,abs_dis
0,57,1.589757,1.589757
1,58,4.504881,4.504881
2,59,5.043841,5.043841
3,60,0.321443,0.321443
4,61,-2.443185,2.443185
...,...,...,...
90,263,-2.013906,2.013906
91,264,1.312743,1.312743
92,265,6.327248,6.327248
93,266,5.622217,5.622217


In [157]:
outdf_CB

,atom_name,residue_name,residue_number,x_coord,y_coord,z_coord
3,CB,ARG,1,-19.548,0.117,-0.729
14,CB,GLN,2,-17.363,-4.088,-3.220
23,CB,GLN,3,-12.863,-2.928,-0.537
32,CB,LEU,4,-15.576,-1.252,3.533
40,CB,TYR,5,-19.833,-4.008,3.599
...,...,...,...,...,...,...
800,CB,ARG,91,-14.250,-19.722,5.362
811,CB,LYS,92,-16.204,-18.073,0.034
820,CB,LEU,93,-10.963,-17.682,-2.362
828,CB,ILE,94,-15.660,-16.372,-4.713


In [175]:
DF = pd.merge(outdf_CB, dis_dic, on=['x_coord', 'y_coord', 'z_coord'], how='inner')

In [177]:
DF.sort_values("abs_dis")

,atom_name,residue_name,residue_number,x_coord,y_coord,z_coord,dis,abs_dis
61,CB,ARG,62,-4.922,3.598,11.930,0.126253,0.126253
78,CB,PRO,79,-2.725,-12.217,11.595,0.281960,0.281960
3,CB,LEU,4,-15.576,-1.252,3.533,0.321443,0.321443
45,CB,ASN,46,-15.713,18.835,5.566,0.360752,0.360752
67,CB,ARG,68,-11.126,-3.970,7.312,-0.397019,0.397019
...,...,...,...,...,...,...,...,...
28,CB,GLU,29,-4.860,6.760,2.268,8.265109,8.265109
73,CB,GLU,74,-7.086,-2.624,20.430,-8.559560,8.559560
24,CB,ILE,25,-13.981,13.947,-4.079,8.746975,8.746975
27,CB,LEU,28,-4.346,12.173,2.220,9.077787,9.077787
